In [ ]:
import numpy as np
import pandas as pd
import sklearn
from tqdm import tqdm
from math import log
from sklearn.model_selection import train_test_split

In [ ]:
import os
os.listdir('/content')
os.path.exists(nyt_path)

True

In [ ]:
# load data
from google.colab import drive
drive.mount('/content/drive')
nyt_path = "/content/nyt.csv"
ag_path = "/content/ag.csv"
nyt_df = pd.read_csv(nyt_path,on_bad_lines='skip')
ag_df = pd.read_csv(ag_path,on_bad_lines='skip')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nyt_df.head()

,text,label
0,(reuters) - carlos tevez sealed his move to ju...,sports
1,if professional pride and strong defiance can ...,sports
2,"palermo, sicily — roberta vinci beat top-seede...",sports
3,spain's big two soccer teams face a pair of it...,sports
4,the argentine soccer club san lorenzo complete...,sports


In [ ]:
# download nltk packages and get stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
print(stop)

{'mustn', 'our', 'both', 'shan', 'you', 'why', 'most', "doesn't", 'for', 'what', 'weren', 'will', "you'll", 'does', 'wouldn', "wasn't", 't', 'm', 'o', 'y', 'itself', 'did', 'against', 'those', 'them', 'the', 'very', "it's", 'further', 'themselves', 'between', 'have', 'it', 'if', 'was', 'down', 'of', 'your', 'be', 'during', 'other', 'ours', 'more', "hasn't", 'to', 'there', 'she', 'while', 'd', 'shouldn', 'these', "should've", 'but', 'had', 'how', 've', "wouldn't", 'their', 'being', 'couldn', 'is', 'only', "haven't", "that'll", "don't", 'so', 'theirs', 'by', 'from', 'own', 'as', 'are', 're', 'at', 'aren', "hadn't", 'doesn', 'doing', 'my', 'an', 'here', "didn't", 'whom', 'through', 'hadn', 'where', "isn't", 'they', 'up', 'haven', 'don', 'when', 'above', 'won', 'he', 'who', 'because', 'then', 'his', "couldn't", "mightn't", 'yourself', 'once', 'ourselves', 'should', 'a', 'her', "she's", 's', 'no', 'isn', "shouldn't", "you've", 'didn', 'am', "won't", 'all', 'until', 'just', 'off', 'has', 'he

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
# define preprocessing function

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer

ps = PorterStemmer()

# return a list of tokens
def pre_processing_by_nltk(doc, stemming = True, need_sent = False):
    # step 1: get sentences
    sentences = sent_tokenize(doc)
    # step 2: get tokens
    tokens = []
    for sent in sentences:
        words = word_tokenize(sent)
        # step 3: stemming
        if stemming:
            words = [ps.stem(word) for word in words if word.lower() not in stop]
        else:
            words = [word for word in words if word.lower() not in stop]
        if need_sent:
            tokens.append(words)
        else:
            tokens += words
    return [w.lower() for w in tokens]

In [ ]:
# Compute token frequency over the corpus, prepare for the next step

from tqdm import tqdm
from collections import defaultdict

DF = defaultdict(float)
for doc in tqdm(nyt_df.text):
    tokens = pre_processing_by_nltk(doc)
    for token in set(tokens):
        DF[token] += 1

100%|██████████| 17534/17534 [05:49<00:00, 50.12it/s]


In [ ]:
# Construct IDF and vocabulary dictionary

IDF, vocab = dict(), dict()
for token in DF:
    if DF[token] < 50:
        # this becomes an unk
        pass
    else:
        vocab[token] = len(vocab) # gives an ID to the token
        IDF[token] = 1 + log(len(nyt_df.text) / DF[token])

# Add unk token to vocab
IDF['<UNK>'] = 1
vocab['<UNK>'] = len(vocab)
print(len(DF), len(vocab))

123922 7784


In [ ]:
# TF-IDF feature extractor function
def tfidf_feature_extractor(doc, vocab, IDF):
    tokens = pre_processing_by_nltk(doc)
    for i, token in enumerate(tokens):
        if token not in vocab:
            tokens[i] = '<UNK>'
    TF = defaultdict(int)
    for token in tokens:
        TF[token] += 1
    x = [0] * len(vocab)
    for token in set(tokens):
        tfidf = log(TF[token] + 1) * IDF[token]
        token_id = vocab[token]
        x[token_id] = tfidf # this will be a dense matrix
    return x

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# Extract feature for binary-x1, Frequency-x2, TF-IDF-x3

x1,x2,x3 = [],[],[]
for doc in tqdm(nyt_df.text):
    tokens = pre_processing_by_nltk(doc)
    v1,v2 = [0]*len(vocab),[0]*len(vocab)
    for token in tokens:
        if token not in vocab:
            v1[-1] = 1
            v2[-1] += 1
        else:
            i = vocab[token]
            v1[i] = 1
            v2[i] += 1
    v3 = tfidf_feature_extractor(doc, vocab, IDF)
    x1.append(v1)
    x2.append(v2)
    x3.append(v3)

100%|██████████| 11519/11519 [06:36<00:00, 29.09it/s]


In [ ]:
y = nyt_df.label

In [ ]:
# Split dataset
x1_train,x1_test_val,y1_train,y1_test_val = train_test_split(x1,y,test_size=0.2,random_state=42)
x1_test,x1_val,y1_test,y1_val = train_test_split(x1_test_val,y1_test_val,test_size=0.5,random_state=42)

x2_train,x2_test_val,y2_train,y2_test_val = train_test_split(x2,y,test_size=0.2,random_state=42)
x2_test,x2_val,y2_test,y2_val = train_test_split(x2_test_val,y2_test_val,test_size=0.5,random_state=42)

x3_train,x3_test_val,y3_train,y3_test_val = train_test_split(x3,y,test_size=0.2,random_state=42)
x3_test,x3_val,y3_test,y3_val = train_test_split(x3_test_val,y3_test_val,test_size=0.5,random_state=42)

In [ ]:
# Train model for (a) Binary
from sklearn.linear_model import LogisticRegressionCV
clf1 = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=42,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(x1_train, y1_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.3min finished


In [ ]:
# Train model for (b) Frequency
clf2 = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=42,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(x2_train, y2_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 14.5min finished


In [ ]:
# Train model for (c) TF-IDF
clf3 = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=42,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(x3_train, y3_train)

In [ ]:
# Analyze function
from sklearn.metrics import f1_score,accuracy_score
def analyze(clf,x_test,y_test):
    y_pred = clf.predict(x_test)
    f1_macro = f1_score(y_test,y_pred,average='macro')
    f1_micro = f1_score(y_test,y_pred,average='micro')
    accuracy = accuracy_score(y_test,y_pred)
    print(f"accuracy={accuracy:.3f}, macro f1={f1_macro:.3f}, micro f1={f1_micro:.3f}")
    return accuracy,f1_macro,f1_micro

In [ ]:
q1_res = pd.DataFrame(columns=['Accuracy','Macro F1','Micro F1'])
q1_res.loc['Binary']=analyze(clf1,x1_test,y1_test)
q1_res.loc['Frequency']=analyze(clf2,x2_test,y2_test)
q1_res.loc['TF-IDF']=analyze(clf3,x3_test,y3_test)

accuracy=0.983, macro f1=0.959, micro f1=0.983
accuracy=0.987, macro f1=0.969, micro f1=0.987
accuracy=0.984, macro f1=0.964, micro f1=0.984


In [ ]:
q1_res

,Accuracy,Macro F1,Micro F1
Binary,0.982639,0.958845,0.982639
Frequency,0.986979,0.969066,0.986979
TF-IDF,0.984375,0.964175,0.984375


**Analysis:**

All three methods achieve pretty well performance (Above 98% accuracy). Besides, we can also observe that both Frequency and TF-IDF features slightly outperform Binary feature, because they encode more information: Frequency and TF-IDF take into account the number of occurance of a word, whereas Binary feature only considers the presence of word in a document.
Also, the fact that Frequency has slightly better performance than TF-IDF suggests that more complicated feature does not always imply better results.

# 2.Word2Vec

In [61]:
import re
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = "/content/glove.6B.100d.txt"
tmp_file = get_tmpfile("test_word2vec.txt")

# Pre-process the GloVe file to ensure proper spacing and handle scientific notation
# and ensure the line has the correct number of dimensions
with open(glove_file, 'r') as infile, open(tmp_file, 'w') as outfile:
    for line in infile:
        parts = line.strip().split()
        # Check if the line has the expected number of dimensions (word + 100 values)
        if len(parts) == 101:
            #  Replace hyphens only if they are NOT preceded by a space AND NOT within a number
            line = re.sub(r'(?<=[^\s\d])-(?=\d)', ' -', line)
            # Ensure scientific notation values are properly formatted
            line = re.sub(r'e(?=[+-])', 'e ', line)  # Insert a space after 'e' before the sign
            line = re.sub(r'e\s+', 'e', line)  # Remove extra spaces, if any, that were present after 'e'

            outfile.write(line)
        else:
            print(f"Skipping line with incorrect dimensions: {line.strip()}")  # Print skipped lines for debugging

# Now use the pre-processed file with glove2word2vec
_ = glove2word2vec(tmp_file, tmp_file + ".word2vec")

model = KeyedVectors.load_word2vec_format(tmp_file + ".word2vec")

Skipping line with incorrect dimensions: standard -0.13416 0.47697 0.45242 0.2767 -0.25912 0.01633 -0.04325 -0.050398 -0.44321 0.77802 0.25418 -0.43745 -0.35909 0.0296 0.23188 -0.2735 0.52555 -0.42301 0.33815 -0.46337 0.21922 -0.57964 0.46917 -0.10207 0.79651 -0.72937 -0.22785 -0.75203 -0.80478 0.46363 0.5083 0.62078 -0.66816 0.020804 1.0387 0.91916 0.32015 0.3477 0.44449 -0.15881 0.01747 -0.489-0.31411 0.46536 1.237 0.32365 0.0062393 -0.13438 -1.0596 0.30352 0.26019 -0.24709 0.34871 -0.15069 0.14109 -0.81446 0.72383 0.31666 -0.12779 0.49355 0.13488 -0.29244 0.75767 -0.33091 -0.30637 -0.19822 1.4267 0.17172 -0.84009 -0.061824 -0.15547 -0.22254 0.21068 0.51183 -0.73276 0.12882 -0.077211 0.30285 0.26494 -0.28881 0.20122 0.53261 -0.081675 0.22325 -0.24023 0.24141 -1.0752 -0.12735 0.60713 -0.86062 0.18556 -0.29058 -0.40533 0.085161 2.3101 0.76856 0.19885 -0.25285 -0.24657 -0.11895 -0.50928 -0.20656 -0.18731 0.26408 -0.26203 0.17246 0.97453 -0.89682 1.4024 0.16583 0.64432 -0.20774 -0.82828 

<ipython-input-61-a5f7c3d75c86>:28: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(tmp_file, tmp_file + ".word2vec")


ValueError: could not broadcast input array from shape (100,) into shape (99,)

In [ ]:
# Train Word2Vec (using gensim package) on AGNews/NYT text data
import pandas as pd
import re
import numpy as np
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
import nltk

nltk.download('punkt')

nltk.download('stopwords') # <--- this is new
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
ps = PorterStemmer()

def pre_processing_by_nltk(doc, stemming = True, need_sent = False):
    # step 1: get sentences
    sentences = sent_tokenize(doc)
    # step 2: get tokens
    tokens = []
    for sent in sentences:
        words = word_tokenize(sent)
        # step 3 (optional): stemming
        if stemming:
            words = [ps.stem(word) for word in words if word.lower() not in stop]
        else:
            words = [word for word in words if word.lower() not in stop]
        if need_sent:
            tokens.append(words)
        else:
            tokens += words
    return [w.lower() for w in tokens]


ag_path = r"C:\Users\DELL\Desktop\DSC 253\HW-1\HW-1\ag.csv"
nyt_path = r"C:\Users\DELL\Desktop\DSC 253\HW-1\HW-1\nyt.csv"
ag_df = pd.read_csv(ag_path)
ag_df['tokens'] = ag_df['text'].apply(pre_processing_by_nltk)
nyt_df = pd.read_csv(nyt_path)
nyt_df['tokens'] = nyt_df['text'].apply(pre_processing_by_nltk)

word2vec_model_ag = Word2Vec(sentences=ag_df['tokens'], vector_size=100, window=5, min_count=2, sg=1, workers=4)
word2vec_model_nyt = Word2Vec(sentences=nyt_df['tokens'], vector_size=100, window=5, min_count=2, sg=1, workers=4)

In [ ]:
# Function to compute document vectors by averaging word vectors
def document_vector(tokens,word2vec_model):
    valid_words = [word for word in tokens if word in word2vec_model.wv]
    if valid_words:
        return np.mean(word2vec_model.wv[valid_words], axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

def document_vector_ag(tokens):
    return document_vector(tokens,word2vec_model_ag)

def document_vector_nyt(tokens):
    return document_vector(tokens,word2vec_model_nyt)

def document_vector_glove(tokens):
    valid_words = [word for word in tokens if word in model]
    if valid_words:
        return np.mean(model[valid_words], axis=0)
    else:
        return np.zeros(model.vector_size)

# Compute document vectors for all documents
nyt_df['doc_vector_ag'] = nyt_df['tokens'].apply(document_vector_ag)
nyt_df['doc_vector_nyt'] = nyt_df['tokens'].apply(document_vector_nyt)
nyt_df['doc_vector_glove'] = nyt_df['tokens'].apply(document_vector_nyt)

# Extract features and labels for further use
x_ag = np.vstack(nyt_df['doc_vector_ag'].values)
x_nyt = np.vstack(nyt_df['doc_vector_nyt'].values)
x_glove = np.vstack(nyt_df['doc_vector_glove'].values)

In [ ]:
X1_train,X1_test_val,Y1_train,Y1_test_val = train_test_split(x_glove,y,test_size=0.2,random_state=42)
X1_test,X1_val,Y1_test,Y1_val = train_test_split(X1_test_val,Y1_test_val,test_size=0.5,random_state=42)

X2_train,X2_test_val,Y2_train,Y2_test_val = train_test_split(x_ag,y,test_size=0.2,random_state=42)
X2_test,X2_val,Y2_test,Y2_val = train_test_split(X2_test_val,Y2_test_val,test_size=0.5,random_state=42)

X3_train,X3_test_val,Y3_train,Y3_test_val = train_test_split(x_nyt,y,test_size=0.2,random_state=42)
X3_test,X3_val,Y3_test,Y3_val = train_test_split(X3_test_val,Y3_test_val,test_size=0.5,random_state=42)

In [ ]:
CLF1 = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=42,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(X1_train, Y1_train)

In [ ]:
CLF2 = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=42,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(X2_train, Y2_train)

In [ ]:
CLF3 = LogisticRegressionCV(cv=5,
                           scoring='accuracy',
                           random_state=42,
                           n_jobs=-1,
                           verbose=3,
                           max_iter=300).fit(X3_train, Y3_train)

In [ ]:
analyze(CLF1,X1_test,Y1_test)

In [ ]:
analyze(CLF2,X2_test,Y2_test)

In [ ]:
analyze(CLF3,X3_test,Y3_test)

**Analysis:**

(a) Word2Vec trained on NYT performs the best across all metrics. This is expected since the Word2Vec model was trained specifically on the NYT dataset, which matches the domain of the test set. The word vectors are highly tailored to this particular dataset, making them more effective in capturing the nuances of the text.
Pre-trained GloVe embeddings also perform very well, but slightly below (iii). GloVe embeddings are trained on a large, diverse corpus, which allows them to generalize well across a wide range of tasks. However, because they are not specifically tuned to the NYT dataset, they fall slightly behind the NYT-specific Word2Vec vectors in performance.
Word2Vec trained on AGNews shows slightly lower performance compared to (i) and (iii). Since the model is trained on a different domain (AGNews) than the NYT test set, the word vectors may not capture the context and language patterns as effectively for this task. Therefore, its performance is slightly lower compared to both the general-purpose GloVe embeddings and the NYT-specific Word2Vec embeddings.

(b) Disadvantages: Firstly, averaging does not preserve the word order or capture important syntactic structures, which can be crucial for understanding the meaning of a sentence.Besides, averaging treats all words equally, without accounting for the varying importance of words in a document. Rare or contextually significant words are given the same weight as common or less important ones.
Idea to Overcome:
For the second disadvantage, one idea to overcome is to use a weighted averaging approach where each word's vector is weighted by its importance in the document, such as TF-IDF. In this case, words that are more relevant to the document have a higher influence on the final vector. This can balance the contribution of frequent words versus rare, but contextually important words.

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_scheduler
from datasets import load_dataset

# Load pretrained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(nyt_df.label.unique()))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Convert label from string to int
label_dic = dict()
for i,label in enumerate(nyt_df.label.unique()):
    label_dic[label] = i
nyt_df['label'] = nyt_df['label'].apply(lambda x:label_dic[x])

In [ ]:
# Tokenization
from datasets import Dataset

dataset = Dataset.from_pandas(nyt_df[['text','label']])
def tokenize_function(df):
    return tokenizer(df['text'], padding='max_length', truncation=True, max_length=64)

tokenized_datasets_raw = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/17534 [00:00<?, ? examples/s]

In [ ]:
# Split Dataset into train, test, validation
from datasets import DatasetDict
train_testvalid = tokenized_datasets_raw.train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
tokenized_datasets = DatasetDict({
'train': train_testvalid['train'],
'test': test_valid['test'],
'valid': test_valid['train']})

In [ ]:
print(tokenized_datasets['train'].features)

{'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Create Dataloader for the convience of training
tokenized_datasets.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
train_dataloader = DataLoader(tokenized_datasets['train'], batch_size=16, shuffle=True)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size=16)

# use GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer and scheduler setup
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(name='linear', optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}  # Move batch to device
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['label'])
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()


In [ ]:
# Evaluation function to compute accuracy, macro F1, and micro F1
from sklearn.metrics import accuracy_score, f1_score
def evaluate(model, dataloader):
    model.eval()
    true_labels, predictions = [], []

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['label'])
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch['label'].cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    macro_f1 = f1_score(true_labels, predictions, average='macro')
    micro_f1 = f1_score(true_labels, predictions, average='micro')

    return accuracy, macro_f1, micro_f1

# Evaluate the model on the test set
accuracy, macro_f1, micro_f1 = evaluate(model, test_dataloader)

print(f"Accuracy: {accuracy:.3f}")
print(f"Macro F1-Score: {macro_f1:.3f}")
print(f"Micro F1-Score: {micro_f1:.3f}")

Accuracy: 0.989
Macro F1-Score: 0.977
Micro F1-Score: 0.989


**Analysis:**

The results show that BERT achieves strong performance on the text classification task, with an accuracy of 0.989, macro F1-score of 0.977, and micro F1-score of 0.989. This indicates that the model is highly effective in both overall accuracy and class-wise balance (macro F1), with minimal discrepancy between classes.
However, there is one significant drawback of BERT compared to previous methods that its training is highly computationally intensive.